In [1]:
import os.path
import pandas as pd


from sklearn.feature_extraction.text import CountVectorizer
import requests

In [2]:
source = pd.read_csv("data/langmeta.csv").fillna(value = "")
stop_words_german = []
with open("data/stop_words_german.txt") as file:
    for line in file:
        stop_words_german.append(line.replace('\n', '').encode('ASCII','ignore').decode('ASCII'))

In [3]:
if (os.path.exists("data/keywordsmeta.csv")):
    os.remove("data/keywordsmeta.csv")

In [4]:

german_vec = CountVectorizer(
    ngram_range=(1, 1), 
    stop_words=stop_words_german,
    max_features=5,
    strip_accents='ascii'
)
english_vec = CountVectorizer(
    ngram_range=(1, 1), 
    stop_words="english",
    max_features=5,
    strip_accents='ascii'
)
#def process_text(text):
#    text = str(text).lower()
#    text = re.sub(
#        f"[{re.escape(string.punctuation)}]", " ", text
#    )
#    text = " ".join(text.split())
#    return text
#
#df["clean_text"] = pd.DataFrame.text.map(process_text)

In [5]:

text=[]
for index, row in source.iterrows():  
    metadict = {"Identifier":row["Identifier"]}      
    #pfad = "temp/" + row["Identifier"] + ".pdf"
    if(row['Language'] != ""):
        if(int(row['Textsize']) > 100):
            if(row['Language'] == "GERMAN"):
                textpfad = "temp/" + row["Identifier"] + ".txt"
                headertextpfad = "temp/" + row["Identifier"] + "_header.txt"
                with open(textpfad, 'r', encoding="utf-8") as textfile:
                    german_vec.fit_transform(textfile)
                    metadict.update({"Topwords":str(german_vec.get_feature_names_out())}) 
                try:
                    with open(headertextpfad, 'r', encoding="utf-8") as textfile:
                        german_vec.fit_transform(textfile.translate(translation_table))
                        metadict.update({"Header_Topwords":str(german_vec.get_feature_names_out())})
                except:
                    print("Error with Headertext")
            elif(row['Language'] == "ENGLISH"):
                textpfad = "temp/" + row["Identifier"] + ".txt"
                with open(textpfad, 'r', encoding="utf-8") as textfile:
                    english_vec.fit_transform(textfile)                    
                    metadict.update({"Topwords":str(english_vec.get_feature_names_out())}) 
                try:
                    with open(headertextpfad, 'r', encoding="utf-8") as textfile:
                        english_vec.fit_transform(textfile)                    
                        metadict.update({"Header_Topwords":str(english_vec.get_feature_names_out())}) 
                except:
                    print("Error with Headertext")
            else:
                frame = pd.DataFrame(metadict, [0])
                if (os.path.exists("data/keywordsmeta.csv")):   
                    #oldframe = pd.read_csv("data/langmeta.csv")
                    newframe = pd.concat([oldframe, frame])
                    newframe.to_csv("data/keywordsmeta.csv", index=False)
                    oldframe = newframe
                else:
                    frame.to_csv("data/keywordsmeta.csv", index=False)
                    oldframe = frame
                metadict.clear
                topwords=""
                print("finished: ", row["Identifier"], " couldn't match language")
                continue
         #ask lobid.org to find a fitting keyword 
            if("Header_Topwords" in metadict):
                wordcloud = str(metadict["Header_Topwords"]).replace("'", "")\
                                                            .removeprefix("[")\
                                                            .removesuffix("]")\
                                                            .split(" ")
            else:
                wordcloud = str(metadict["Topwords"]).replace("'", "")\
                                                     .removeprefix("[")\
                                                     .removesuffix("]")\
                                                     .split(" ")
            Keywordlist= list()
            KeyIDlist =list()
            for word in wordcloud:
                api_url = "https://lobid.org/gnd/search?q=preferredName:" + word + "&filter=type:SubjectHeading&format=json"
                #print(api_url)
                response = requests.get(api_url)
                if(response.status_code == requests.codes.ok):
                    for member in response.json()["member"]: 
                        if('SubjectHeading' in member["type"] or 'SubjectHeadingSensoStricto' in member["type"]):
                            Keywordlist.append(member["preferredName"])
                            KeyIDlist.append(member["id"])
                            #metadict.update({"KeywordID":})
                            #metadict.update({"Keywordname":member["preferredName"]})
                            #foundstandard = True
                            break
                #if(foundstandard):
                #    break
            metadict.update({"Keywordnames":str(Keywordlist)})       
            metadict.update({"KeywordIDs":str(KeyIDlist)})                           
                    
                    
                    
                    
                    
                    
        frame = pd.DataFrame(metadict, [0])
        if (os.path.exists("data/keywordsmeta.csv")):   
        #oldframe = pd.read_csv("data/langmeta.csv")
            newframe = pd.concat([oldframe, frame])
            newframe.to_csv("data/keywordsmeta.csv", index=False)
            oldframe = newframe
        else:
            frame.to_csv("data/keywordsmeta.csv", index=False)
            oldframe = frame
    metadict.clear
    topwords=""
    #foundstandard = False 
    print("finished: ", row["Identifier"])
    #break     


C:\Users\Johann\.virtualenvs\pipenv-5hQiS9Jl\lib\site-packages\sklearn\feature_extraction\text.py:404: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ernst', 'ordnung', 'schluss', 'wahr'] not in stop_words.
  warnings.warn(


Error with Headertext
finished:  8I6sM5zapD60
finished:  8ZICOHBmAHyQ
Error with Headertext
finished:  3ztCv-WpxJ4U
Error with Headertext
finished:  6mOhjfscZK2A
Error with Headertext
finished:  1eteONeHL82Y
Error with Headertext
finished:  1mjbqKfwSW7U
Error with Headertext
finished:  1BruMQFjEIRY
Error with Headertext
finished:  1hZf0CFwsBhA
Error with Headertext
finished:  1PrtXFruGers
Error with Headertext
finished:  1LvS8_gxjyYc
finished:  18r_JtdoG2RA
finished:  1zfzKOhH3Szw
finished:  1LtI4gnpDFIQ
Error with Headertext
finished:  17pFIynqIjGc
Error with Headertext
finished:  1Xamc2rgc3UA
Error with Headertext
finished:  1055-D7j6GpXg
Error with Headertext
finished:  9kfa9dR1C5xw
Error with Headertext
finished:  1lWB4XHk4REw
finished:  1R6Y6mMha5-E
Error with Headertext
finished:  11r0NIcRyxxo
Error with Headertext
finished:  19CzcUUPmFeI
finished:  1_RecnQsU3os
Error with Headertext
finished:  1CnMqavsnGA0
Error with Headertext
finished:  1fCUEOJoo3Hg
Error with Headertext
finis

finished:  4P2wkFjguZB0
Error with Headertext
finished:  9h_vYcI6Oin4
Error with Headertext
finished:  7UNFOSSE6qy0
Error with Headertext
finished:  7pz2gdOsZ4X8
Error with Headertext
finished:  42VnByBSUYOc
Error with Headertext
finished:  3rbQErjj5aOM
Error with Headertext
finished:  5vB7Ie4mYqAw
Error with Headertext
finished:  1DW-hA_COeOk
Error with Headertext
finished:  9fepMllDAq9Q
Error with Headertext
finished:  5xEbJkJSjzfg
Error with Headertext
finished:  5tG0b0gUEDBM
Error with Headertext
finished:  8z_j3yTnMllU
finished:  7Y-VydusVzi0
finished:  1yZiGS-7wOOc
finished:  1ZYbXnJbhMUw
Error with Headertext
finished:  443WHOokp1Uk
finished:  9cO2tJoUC0xg
Error with Headertext
finished:  7WPzWJsRHeJE
Error with Headertext
finished:  7uvzrjnzQ0LM
Error with Headertext
finished:  9DDwalGCh3xc
Error with Headertext
finished:  4BXI8pXYmUCA
Error with Headertext
finished:  3SnQXTRtAwHM
Error with Headertext
finished:  89RRqLYq0fIE
Error with Headertext
finished:  38uPV4VdYK1s
Error 

finished:  3JBYu-LasOec
Error with Headertext
finished:  8oqPTtA_MQq0
Error with Headertext
finished:  5OktWJqUFKnA
Error with Headertext
finished:  7WUTZQuDyTYQ
Error with Headertext
finished:  4Bbc9lr4oON8
Error with Headertext
finished:  9fRB_2YsCmHQ
finished:  107DNRnpjqloU
Error with Headertext
finished:  3Sl3ty-0ZMmU
Error with Headertext
finished:  55HVfvDJiisk
Error with Headertext
finished:  30DzqhF6PTqQ
finished:  8Jm1Lu2X69sg
finished:  3U-qpzV7WItY
finished:  3_h6upB-apDo
Error with Headertext
finished:  7p1uZJa85YMk
Error with Headertext
finished:  9ueURG74Vx5g
Error with Headertext
finished:  6_73AyhObIaM
Error with Headertext
finished:  522R86HcIm3Q
finished:  4K-OYhppC_iA
Error with Headertext
finished:  5RGIOrbMT-X4
Error with Headertext
finished:  4F4_5JL0s-wo
Error with Headertext
finished:  4EqglpdgG2oM
finished:  6_O4JR41hsWY
finished:  6F1NkzSSt-3A
finished:  8ozGDFYLGLHo  couldn't match language
Error with Headertext
finished:  8VvKeuUSFWjY
Error with Headertext


finished:  1zEPVyh-0Qbo
Error with Headertext
finished:  1tdf8LZ4p8YI
Error with Headertext
finished:  3N-Fs0EC36ZM
Error with Headertext
finished:  1v3cq87o5dD4
Error with Headertext
finished:  1O8vVhEoD51o
Error with Headertext
finished:  10CyCH3sJD5v0
Error with Headertext
finished:  1VEUIv5jqZ8M
Error with Headertext
finished:  1gnG44_FmP1E
finished:  18CC6M_NM_Tg
Error with Headertext
finished:  1xijumXcYTbg
finished:  10Rq2FvCe_-Y
Error with Headertext
finished:  1QWWxuPmAdEc
Error with Headertext
finished:  1NX924Kq5Opo
Error with Headertext
finished:  1Tc-LuexbpL0
Error with Headertext
finished:  1oL8zg2l0RvA
Error with Headertext
finished:  103uXAouHOAE
Error with Headertext
finished:  1HAYdXCAqRZk
Error with Headertext
finished:  1aFnLKYjeS-o
finished:  1x-dAQDh27JE  couldn't match language
Error with Headertext
finished:  11YWMGNugj9k
Error with Headertext
finished:  1o1vxar3rTlw
Error with Headertext
finished:  12qoVGCqjiTwo
finished:  10Scu4roiqRYA  couldn't match language

...

finished:  11DkJZZ1scNjI
Error with Headertext
finished:  11XGIm65r9r8c
Error with Headertext
finished:  81Q_Ai3OIQ7o
Error with Headertext
finished:  12oEw9-aEbvwM
Error with Headertext
finished:  1237YBKoYyxgw
Error with Headertext
finished:  120P2pFLjBfXg
Error with Headertext
finished:  12IBEs76n95Ek
Error with Headertext
finished:  120LGX7bIParM
Error with Headertext
finished:  121qFmEuuAWQY
Error with Headertext
finished:  127wDDblYRlNQ
Error with Headertext
finished:  11axDTkQDG4kw
finished:  11h1GqJrkXP2w
Error with Headertext
finished:  11i1Vn_n0B4OE
Error with Headertext
finished:  11DwbyxtY6AO0
finished:  11WTV6LCBCReQ
Error with Headertext
finished:  11MZWZxjZNyrw
finished:  11lX_y24-bAjw
finished:  11jRqFhVjNtnc
finished:  1143eQF1qpymI  couldn't match language
finished:  11H8gOBeToCzw  couldn't match language
Error with Headertext
finished:  11TXv-hhS4WRM
finished:  11r2kRYV7S7V4  couldn't match language
Error with Headertext
finished:  11slcUMDM45vE
Error with Headertext

finished:  8hJOwmRfa5_A
Error with Headertext
finished:  8FlmFo_vHcTs
Error with Headertext
finished:  8v8nHYhuzBew
Error with Headertext
finished:  8yE67I_9-tRc
Error with Headertext
finished:  8cXpUtYPMK40
Error with Headertext
finished:  85aJX5qRVwXA
Error with Headertext
finished:  8qezXQkYsluU
Error with Headertext
finished:  8wU9GcAJip-M
Error with Headertext
finished:  8ega6oBifzXc
Error with Headertext
finished:  8Tb6TwP5YP4k
Error with Headertext
finished:  8_HUABVzcCXw
Error with Headertext
finished:  8b0Vj7p6azmQ
Error with Headertext
finished:  8i6Hxlg30Fw8
Error with Headertext
finished:  8ELCxSR_QNdI
Error with Headertext
finished:  8sTG35VRIK7Q
Error with Headertext
finished:  8bptTYZu-htg
Error with Headertext
finished:  8T6UxODlwsgk
Error with Headertext
finished:  8Na1srLxFFI4
Error with Headertext
finished:  8V5d8MGKMOrM
Error with Headertext
finished:  80UzCm7gNvvU
Error with Headertext
finished:  8lOHWyiPsy4g
Error with Headertext
finished:  8pLiVHfDZvFY
Error with

finished:  5CE_OIelucKY
Error with Headertext
finished:  5goTmlGVcAZ4
finished:  37uHtzYkPsI0
Error with Headertext
finished:  3M1h86MYoYO0
Error with Headertext
finished:  110Gey3Q-6kvU
Error with Headertext
finished:  9zTE8XxsXBm0
Error with Headertext
finished:  859UwxlupCDo
Error with Headertext
finished:  83Lod-nW4H7Y
Error with Headertext
finished:  8TIRsj9pQkz4
Error with Headertext
finished:  8oEv0wa-W3BY
finished:  8lH8tFZtPYfc
finished:  8mjb5kfkkL-M
Error with Headertext
finished:  3hfZchCVhRNE
Error with Headertext
finished:  3xkZurCuel1o
Error with Headertext
finished:  3UKH92tA2C8M
Error with Headertext
finished:  3h1VZ344QH88
Error with Headertext
finished:  3vJLFG3k5_v0
Error with Headertext
finished:  3s-FmM2ctJsE
Error with Headertext
finished:  3nlF77E1S290
Error with Headertext
finished:  37rAM1I1GH-E
Error with Headertext
finished:  3niObKlcbgn8
Error with Headertext
finished:  38ZwtuZjvDg4
Error with Headertext
finished:  3Pn3jimLzuQs
Error with Headertext
finishe

finished:  10pyfgWQc9Gzk
Error with Headertext
finished:  108xWlGul3UoA
Error with Headertext
finished:  10ZEPc99l_yGk
Error with Headertext
finished:  10FVro6fjMGGw
Error with Headertext
finished:  10HLDpcnPTMnA
Error with Headertext
finished:  106b_RitP6y4Y
Error with Headertext
finished:  10r_cSFqAmbqQ
Error with Headertext
finished:  10KmOiNH3Xvug
Error with Headertext
finished:  10Ny13rJarhrs
Error with Headertext
finished:  10s_iYOs-bNZo
Error with Headertext
finished:  102ZZmNntpZuE
Error with Headertext
finished:  10nhRMXYbhxFo
Error with Headertext
finished:  10Il3BCwusDos
Error with Headertext
finished:  10yJKWU72U_mA
Error with Headertext
finished:  10Vpeyu-FaYOg
Error with Headertext
finished:  10pU_koiqdmkk
Error with Headertext
finished:  10eU9WPCjZm6M
Error with Headertext
finished:  10t17oE4o2r3Q
finished:  10VZL6Ye1iAYE
Error with Headertext
finished:  10HvdJWmJHTZQ
Error with Headertext
finished:  10AOY20yvtKi0
Error with Headertext
finished:  10RHb_9AFGRI8
Error with

finished:  12-w__zhYO71o
Error with Headertext
finished:  5s5fLCflWxiQ
finished:  4CirJUu0t4YA
finished:  4QV7cUfpfGdM
Error with Headertext
finished:  5J-jhJoWhpm4
Error with Headertext
finished:  11Tl17LXsoaek
Error with Headertext
finished:  10FxaP3vJ4SKY
finished:  9FfZo4HNnL8M
Error with Headertext
finished:  6qsAtQBscQQs
Error with Headertext
finished:  6i2kpBkY54Qg
finished:  6N5cVnJwdaWU
finished:  6_3VLYSwx8YA
Error with Headertext
finished:  118-kl8Rkeg7Y
finished:  8l1Ed1xPgPr8
Error with Headertext
finished:  6AYgdCiQxIYM
Error with Headertext
finished:  8RudnuIuYARg
Error with Headertext
finished:  9wUtYyAHlkA4
Error with Headertext
finished:  9ht2A_z0v-ro
Error with Headertext
finished:  9SbAiJCEZz0Q
Error with Headertext
finished:  9mBH90br-HvY
finished:  3EgL0MtD0qmc
finished:  8btwH0ZJcWuA
Error with Headertext
finished:  12YvX9Fx1nJ3k
Error with Headertext
finished:  12CpPiapoDjI8
finished:  9Cve1j97o-a4
finished:  8-pVUd639luM
Error with Headertext
finished:  3ydrEub